In [203]:
import time

import pandas as pd
import requests
from lxml.html import fromstring
from bs4 import BeautifulSoup
from tqdm import tqdm

In [194]:
url = 'https://www.pazar3.mk/ads/real-estate/for-sale/skopje'

In [256]:
page = 1

In [259]:
# url_all = f'https://www.pazar3.mk/ads/real-estate/for-sale?Page={page}'

In [195]:
response = requests.get(url)

In [185]:
response.text

'\r\n<!DOCTYPE html>\n<html class="site-Macedonia" dir="ltr" lang="en-US" prefix="og:http://ogp.me/ns#">\n<head>\n<meta charset="UTF-8">\n<title>Real Estate | Skopje - Pazar3.mk</title>\n\n<meta name="title" content="Real Estate | Skopje - Pazar3.mk" />\n<meta name="description" content="MetaDescription" />\n<meta property="og:title" content="Real Estate | Skopje - Pazar3.mk" />\n<meta property="og:description" content="The biggest number of ads on one place! Have a look at all active ads on Pazar3.mk according to your search | Real Estate | Skopje" />\n<meta property="og:image" content="https://www.pazar3.mk/Content/Images/OpenGraphics/Pazar3_mk.png" />\n<meta property="og:type" content="website" />\n\n<script type="application/ld+json">\r\n                {"@context":"http://schema.org","@graph":[{"@type":"Organization","@id":"https://www.pazar3.mk/#organization","image":"https://www.pazar3.mk/Content/Images/Macedonia/Macedonia.svg","url":"https://www.pazar3.mk","sameAs":["https://pl

In [269]:
hrefs = []
for page in tqdm(range(1, 100)):
    url_all = f'https://www.pazar3.mk/ads/real-estate/for-sale?Page={page}' 
    response = requests.get(url_all)
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
    
        # Find all links with a specific class (like "SearchAdTitle")
        links = soup.find_all('a', class_='Link_vis')
    
        # Extract the href attributes from the links
        hrefs.extend([link.get('href') for link in links])
    
        # Print all the hrefs
        # for href in hrefs:
        #     print(href)
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
print(len(hrefs))

100%|██████████| 99/99 [02:23<00:00,  1.45s/it]

5544


In [268]:
len(hrefs)

112

In [152]:
new_url = 'https://www.pazar3.mk/ad/real-estate/apartments/for-sale/skopje/gjorce-petrov/se-prodava-stan-vo-gjorce-petrov/5837372'

In [153]:
r = requests.get(new_url)

In [154]:
r.status_code

200

In [155]:
soup = BeautifulSoup(r.text, 'html.parser')

In [172]:
title = soup.find('h1', class_='ci-text-base').text

In [173]:
title

'Se prodava stan vo Gjorce Petrov'

In [158]:
price = soup.find('h4', class_='ci-text-success')

In [159]:
price

<h4 class="ci-text-success">
<bdi class="new-price">
<span class="format-money-int" value="102000"></span><span>EUR</span>
</bdi>
</h4>

In [160]:
price.find('span').get('value')

'102000'

In [161]:
price.find_all('span')

[<span class="format-money-int" value="102000"></span>, <span>EUR</span>]

In [162]:
currency = price.find_all('span')[1].text

In [163]:
currency

'EUR'

In [164]:
tags = soup.find_all('div', class_='tags-area')

In [169]:
tags

[<div class="tags-area">
 <a class="tag-item" href="https://www.pazar3.mk/ads/real-estate/apartments/for-sale?Prop=102-3--from-number-of-rooms-2-5">
 <span>Number of rooms:</span>
 <bdi>
 2
 </bdi>
 </a>
 <a class="tag-item" href="https://www.pazar3.mk/ads/real-estate/apartments/for-sale?Prop=146-804-condition-used-like-new">
 <span>Condition:</span>
 <bdi>
 Used - Like New
 </bdi>
 </a>
 <a class="tag-item" href="https://www.pazar3.mk/ads/real-estate/apartments/for-sale?Prop=119-0-area-75">
 <span>Area:</span>
 <bdi>
 75
 m<sup>2</sup>
 </bdi>
 </a>
 <a class="tag-item" href="https://www.pazar3.mk/ads/real-estate/apartments/for-sale"><span>Ad type:</span><bdi>For sale</bdi></a>
 <a class="tag-item" href="https://www.pazar3.mk/ads/real-estate/apartments/for-sale?Private=True">
 <span>Advertised by:</span>
 <bdi>
 Store </bdi>
 </a>
 <a class="tag-item" href="https://www.pazar3.mk/ads/real-estate/apartments/for-sale/skopje/gjorce-petrov"><span>Location:</span><bdi>Gjorce Petrov, Skopje<

In [170]:
for tag in tags:
    # Find all 'a' tags within each 'tags-area' div
    a_tags = tag.find_all('a', class_='tag-item')
    
    # Extract the specific information needed
    for a in a_tags:
        label = a.find('span').text.strip()  # Extract the label
        value = a.find('bdi').text.strip()   # Extract the value

        # Check for the specific tags you want
        if 'Number of rooms' in label:
            number_of_rooms = value
        elif 'Area' in label:
            area = value + " m²"
        elif 'Location' in label:
            location = value

# Print the extracted information
print(f"Number of rooms: {number_of_rooms}")
print(f"Area: {area}")
print(f"Location: {location}")

Number of rooms: 2
Area: 75
m2 m²
Location: Gjorce Petrov, Skopje


In [236]:
df = pd.DataFrame(columns=['Title', 'Price', 'Currency', 'Number_of_rooms', 'Area', 'Location'])

In [270]:
len(hrefs)

5544

In [222]:
def get_property_values(url):
    r = requests.get(url)
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, 'html.parser')

        # Title
        title = soup.find('h1', class_='ci-text-base').text.strip() if soup.find('h1', class_='ci-text-base') else ""

        # Price, Currency
        price_class = soup.find('h4', class_='ci-text-success')
        price = price_class.find('span').get('value') if price_class and price_class.find('span') else ""
        currency = price_class.find_all('span')[1].text.strip() if price_class and len(price_class.find_all('span')) > 1 else ""

        # Initialize the fields
        number_of_rooms = ""
        area = ""
        location = ""

        # Tags Area
        tags = soup.find_all('div', class_='tags-area')
        for tag in tags:
            a_tags = tag.find_all('a', class_='tag-item')

            for a in a_tags:
                label = a.find('span').text.strip() if a.find('span') else ""
                value = a.find('bdi').text.strip() if a.find('bdi') else ""

                if 'Number of rooms' in label:
                    number_of_rooms = value
                elif 'Area' in label:
                    area = value + " m²"
                elif 'Location' in label:
                    location = value
        
        property_data = {
            'Title': title,
            'Price': price,
            'Currency': currency,
            'Number_of_rooms': number_of_rooms,
            'Area': area,
            'Location': location
        }
        
        return property_data
        # Append the data to the DataFrame
        # df._append(property_data, ignore_index=True)
        
        # Print the extracted information
        # print(f"Title: {title}")
        # print(f"Price: {price}")
        # print(f"Currency: {currency}")
        # print(f"Number of rooms: {number_of_rooms}")
        # print(f"Area: {area}")
        # print(f"Location: {location}")
        # print('=' * 10)
    else:
        print(f"Failed to retrieve the page. Status code: {r.status_code}")

In [223]:
print(hrefs)

['/ad/real-estate/apartments/for-sale/skopje/karposh/stan-na-partizanska-nasproti-simpo/5350124', '/ad/real-estate/apartments/for-sale/skopje/gazi-baba/novi-100-stanovi-vo-zhelezara/5364086', '/ad/real-estate/apartments/for-sale/skopje/karposh/stan-od-134m2-vo-karpos/4748641', '/ad/real-estate/apartments/for-sale/skopje/gazi-baba/se-prodava-kukja-vo-zelezara/5837381', '/ad/real-estate/apartments/for-sale/skopje/chair/se-prodava-kukja-na-plasticarska-ulica/5837376', '/ad/real-estate/apartments/for-sale/skopje/chair/se-prodava-stan-vo-cair/5837374', '/ad/real-estate/apartments/for-sale/skopje/gjorce-petrov/se-prodava-stan-vo-gjorce-petrov/5837372', '/ad/real-estate/apartments/for-sale/skopje/centar/se-prodava-stan-na-kale/5837370', '/ad/real-estate/apartments/for-sale/skopje/butel/se-prodava-stan-vo-skopje-sever/5837367', '/ad/real-estate/apartments/for-sale/skopje/aerodrom/stan-59m2-vo-aerodrom/5837366', '/ad/real-estate/apartments/for-sale/skopje/gjorce-petrov/se-prodava-stan-vo-gjorce

In [322]:
dataframe = pd.DataFrame(columns=['Title', 'Price', 'Currency', 'Number_of_rooms', 'Area', 'Location'])

for link in tqdm(hrefs):
    row = get_property_values('https://www.pazar3.mk' + link)
    dataframe = dataframe._append(row, ignore_index=True)
    # time.sleep(1)

  4%|▍         | 224/5544 [01:50<36:37,  2.42it/s]  

Failed to retrieve the page. Status code: 404


  4%|▍         | 245/5544 [01:58<22:39,  3.90it/s]

Failed to retrieve the page. Status code: 404


  6%|▌         | 334/5544 [02:31<27:49,  3.12it/s]

Failed to retrieve the page. Status code: 404


 13%|█▎        | 727/5544 [05:11<25:05,  3.20it/s]

Failed to retrieve the page. Status code: 404


 21%|██        | 1165/5544 [08:16<31:05,  2.35it/s]  


KeyboardInterrupt: 

In [323]:
len(dataframe)

1161

In [324]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Title            1161 non-null   object
 1   Price            1161 non-null   object
 2   Currency         1161 non-null   object
 3   Number_of_rooms  1161 non-null   object
 4   Area             1161 non-null   object
 5   Location         1161 non-null   object
dtypes: object(6)
memory usage: 54.6+ KB


In [325]:
dataframe.Currency.unique()

array(['', 'EUR', 'MKD'], dtype=object)

In [326]:
dataframe['Area(m2)'] = dataframe['Area'].str.extract(r'(\d+)')

In [327]:
dataframe.drop(columns=['Area'], inplace=True)

In [330]:
dataframe

,Title,Price,Currency,Number_of_rooms,Location,Area(m2)
0,Gradezna parcela,,,,"Vasilevo, Strumica",3000
1,Се издава нов стан 76 m2 до ОУ Блаже Конески,550,EUR,4,"Aerodrom, Skopje",76
2,SHITEN SHTEPIT DHE PARCELAT,1,EUR,> 10,Kumanovo,1600
3,Se prodava dupleks vo blizina na drzavna bolnica,105000,EUR,3,"Centar, Skopje",90
4,Renoviran trosoben stan vo Novo Lisice,125000,EUR,3,"Aerodrom, Skopje",72
...,...,...,...,...,...,...
1156,PRODAVAM NIVA,330,EUR,,Sveti Nikole,1700
1157,PRODAVAM NIVA,300,EUR,,Sveti Nikole,3700
1158,СТАН ДУПЛЕКС на елитна локација на Водно со по...,2200,EUR,4,"Centar, Skopje",95
1159,Imperija Nedviznosti kuka so pogled na ezero,105000,EUR,8,Ohrid,175


In [331]:
dataframe['Price'] = pd.to_numeric(dataframe['Price'], errors='coerce')

In [332]:
dataframe['Area(m2)'] = pd.to_numeric(dataframe['Area(m2)'], errors='coerce')

In [333]:
dataframe['Number_of_rooms'] = pd.to_numeric(dataframe['Number_of_rooms'], errors='coerce')

In [334]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Title            1161 non-null   object 
 1   Price            878 non-null    float64
 2   Currency         1161 non-null   object 
 3   Number_of_rooms  826 non-null    float64
 4   Location         1161 non-null   object 
 5   Area(m2)         1085 non-null   float64
dtypes: float64(3), object(3)
memory usage: 54.6+ KB


In [337]:
dataframe

,Title,Price,Currency,Number_of_rooms,Location,Area(m2)
0,Gradezna parcela,NaN,,NaN,"Vasilevo, Strumica",3000.0
1,Се издава нов стан 76 m2 до ОУ Блаже Конески,550.0,EUR,4.0,"Aerodrom, Skopje",76.0
2,SHITEN SHTEPIT DHE PARCELAT,1.0,EUR,NaN,Kumanovo,1600.0
3,Se prodava dupleks vo blizina na drzavna bolnica,105000.0,EUR,3.0,"Centar, Skopje",90.0
4,Renoviran trosoben stan vo Novo Lisice,125000.0,EUR,3.0,"Aerodrom, Skopje",72.0
...,...,...,...,...,...,...
1156,PRODAVAM NIVA,330.0,EUR,NaN,Sveti Nikole,1700.0
1157,PRODAVAM NIVA,300.0,EUR,NaN,Sveti Nikole,3700.0
1158,СТАН ДУПЛЕКС на елитна локација на Водно со по...,2200.0,EUR,4.0,"Centar, Skopje",95.0
1159,Imperija Nedviznosti kuka so pogled na ezero,105000.0,EUR,8.0,Ohrid,175.0


In [338]:
dataframe.to_csv('pazar3_properties.csv', index=False)